###### Imports and Pulls

In [1]:
import pandas as pd
import numpy as np
import requests
#import io
import pickle
from collections import deque
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
pd.options.mode.chained_assignment = None  # default='warn'

###### API Key

In [2]:
#to read in... rb is read bite
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)
#create a variable that contains your api key
api_key = keys_dict_2['CENSUS']

###### Geography Lists

In [3]:
GNRC = ['111', #Macon
       '161', #Stewart
       '125', #Montgomery
       '083', #Houston
       '085', #Humphreys
       '043', #Dickson
       '021', #Cheatham
       '147', #Robertson
       '165', #Sumner
       '037', #Davidson
       '189', #Wilson
       '169', #Trousdale
       '187', #Williamson
       '149', #Rutherford
       '119'] #Maury
KY = ['003', #Allen
      '213'] #Simpson

In [4]:
slaces = ['02180', #Ashland City: Cheatham
          '39660', #Kingston Springs: Cheatham
          '57480', #Pegram: Cheatham
          '59560', #Pleasant View: Cheatham
          '04620', #Belle Meade: Davidson
          '05140', #Berry Hill: Davidson
          '27020', #Forest Hills: Davidson
          '29920', #Goodlettsville: Davidson/Sumner
          '52006', #Nashville-Davidson metropolitan government (balance): Davidson
          '54780', #Oak Hill: Davidson
          '63140', #Ridgetop: Davidson/Robertson
          '09880', #Burns: Dickson
          '13080', #Charlotte: Dickson
          '20620', #Dickson: Dickson
          '69080', #Slayden: Dickson
          '76860', #Vanleer: Dickson
          '79980', #White Bluff: Dickson
          '24320', #Erin: Houston
          '73460', #Tennessee Ridge: Houston/Stewart
          '44840', #McEwen: Humphreys
          '52820', #New Johnsonville: Humphreys
          '78560', #Waverly: Humphreys
          '16540', #Columbia: Maury
          '51080', #Mount Pleasant: Maury
          '70580', #Spring Hill: Maury/Williamson
          '15160', #Clarksville: Montgomery
          '00200', #Adams: Robertson
          '11980', #Cedar Hill: Robertson
          '16980', #Coopertown: Robertson
          '18420', #Cross Plains: Robertson
          '30960', #Greenbrier: Robertson
          '48980', #Millersville: Robertson/Sumner
          '60280', #Portland: Robertson/Sumner
          '70500', #Springfield: Robertson
          '80200', #White House: Robertson/Sumner
          '22360', #Eagleville: Rutherford
          '41200', #La Vergne: Rutherford
          '51560', #Murfreesboro: Rutherford
          '69420', #Smyrna: Rutherford
          '18820', #Cumberland City: Stewart
          '21400', #Dover: Stewart
          '28540', #Gallatin: Sumner
          '33280', #Hendersonville: Sumner
          '79420', #Westmoreland: Sumner
          '08280', #Brentwood: Williamson
          '25440', #Fairview: Williamson
          '27740', #Franklin: Williamson
          '53460', #Nolensville: Williamson
          '73900', #Thompson's Station: Williamson
          '41520', #Lebanon: Wilson
          '50780', #Mount Juliet: Wilson
          '78320', #Watertown: Wilson
          '40160', #Lafayette: Macon
          '69114', #Scottsville city, Kentucky
          '28918', #Franklin city, Kentucky
          '49460'] #Mitchelville city: Sumner

At this time the only variable from the subject tables is "average commute time to work" so there is no data guide. In 2010 and 2020 the variable remains consistent: S0801_C01_046E

In [5]:
#2010 ACS 5 Year Subject Table Estimate
#counties
url_str= 'https://api.census.gov/data/2010/acs/acs5/subject?key='+api_key
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47" 
data = requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS', 'GeoFIPS']
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47" 
data = requests.get(url_str, params= predicates)                                                              
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS', 'GeoFIPS']
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = pd.concat([df, kycos], axis = 0)
#ky places call
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47" 
data = requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS', 'GeoFIPS']
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GeoFIPS'].isin(slaces)]
df = pd.concat([df, places], axis = 0)
#places
url_str= 'https://api.census.gov/data/2010/acs/acs5/subject?key='+api_key
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47" 
data = requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS', 'GeoFIPS']
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GeoFIPS'].isin(slaces)]
df = pd.concat([df, places], axis = 0)
#state call
url_str= 'https://api.census.gov/data/2010/acs/acs5/subject?key='+api_key
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS']
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = pd.concat([df, state], axis = 0)
#national call
url_str= 'https://api.census.gov/data/2010/acs/acs5/subject?key='+api_key
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS']
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [6]:
twentyten = savename

In [7]:
twentyten.tail()

,NAME,GEO_ID,Mean Travel Time to Work,StateFIPS,GeoFIPS
413,"Westmoreland town, Tennessee",1600000US4779420,33.4,47,79420
414,"White Bluff town, Tennessee",1600000US4779980,32.6,47,79980
415,"White House city, Tennessee",1600000US4780200,28.8,47,80200
0,Tennessee,0400000US47,23.9,47,0
0,United States,0100000US,25.2,1,0


In [8]:
#2020 ACS 5 Year Subject Table Estimate
#counties
url_str= 'https://api.census.gov/data/2020/acs/acs5/subject?key='+api_key
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47" 
data = requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS', 'GeoFIPS']
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47" 
data = requests.get(url_str, params= predicates)                                                              
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS', 'GeoFIPS']
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = pd.concat([df, kycos], axis = 0)
#ky places call
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47" 
data = requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS', 'GeoFIPS']
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GeoFIPS'].isin(slaces)]
df = pd.concat([df, places], axis = 0)
#places
url_str= 'https://api.census.gov/data/2010/acs/acs5/subject?key='+api_key
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47" 
data = requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS', 'GeoFIPS']
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GeoFIPS'].isin(slaces)]
df = pd.concat([df, places], axis = 0)
#state call
url_str= 'https://api.census.gov/data/2010/acs/acs5/subject?key='+api_key
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS']
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = pd.concat([df, state], axis = 0)
#national call
url_str= 'https://api.census.gov/data/2010/acs/acs5/subject?key='+api_key
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS']
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [9]:
twentytwenty = savename

In [10]:
twentytwenty.tail()

,NAME,GEO_ID,Mean Travel Time to Work,StateFIPS,GeoFIPS
413,"Westmoreland town, Tennessee",1600000US4779420,33.4,47,79420
414,"White Bluff town, Tennessee",1600000US4779980,32.6,47,79980
415,"White House city, Tennessee",1600000US4780200,28.8,47,80200
0,Tennessee,0400000US47,23.9,47,0
0,United States,0100000US,25.2,1,0


In [10]:
twentyten = twentyten.drop(columns = ['StateFIPS', 'GeoFIPS'])

In [12]:
twentyten.head(3)

,NAME,GEO_ID,Mean Travel Time to Work
1,"Stewart County, Tennessee",0500000US47161,32.0
3,"Sumner County, Tennessee",0500000US47165,27.6
5,"Trousdale County, Tennessee",0500000US47169,31.9


In [13]:
twentyten.to_csv('../../Outputs/2010Subject.csv', index = False)

In [14]:
twentytwenty = twentytwenty.drop(columns = ['StateFIPS', 'GeoFIPS'])

In [15]:
twentytwenty.head()

,NAME,GEO_ID,Mean Travel Time to Work
1,"Montgomery County, Tennessee",0500000US47125,26.2
9,"Rutherford County, Tennessee",0500000US47149,28.8
15,"Sumner County, Tennessee",0500000US47165,29.5
16,"Trousdale County, Tennessee",0500000US47169,41.0
22,"Williamson County, Tennessee",0500000US47187,27.8


In [17]:
twentytwenty.to_csv('../../Outputs/2020Subject.csv', index = False)